# pdfplumber使用说明

- 使用pdfplumber.open打开要处理的PDF文件
- 当PDF受密码保护时需要传递password关键字参数，pdfplumber.open("file.pdf", password = "test")

In [8]:
import pdfplumber

with pdfplumber.open("/Users/greedy/Documents/所有简历数据/IT/1.pdf") as pdf:
    first_page = pdf.pages[0]
    print(first_page.chars[0])

{'fontname': 'WNPYQS+STKaitiSC-Regular', 'adv': 480.0, 'upright': True, 'x0': 89.28, 'y0': 742.758, 'x1': 113.28, 'y1': 766.758, 'width': 24.0, 'height': 24.0, 'size': 24.0, 'object_type': 'char', 'page_number': 1, 'stroking_color': (0, 0, 0), 'non_stroking_color': (0, 0, 0), 'text': '王', 'top': 75.24199999999996, 'bottom': 99.24199999999996, 'doctop': 75.24199999999996}


## 文件相关信息

In [12]:
pdf.metadata

{'Author': '葛二蛋',
 'Comments': '',
 'Company': '',
 'CreationDate': "D:20220117152121+07'21'",
 'Creator': 'WPS 文字',
 'Keywords': '78520e38dc3d33721nRy39y8EFtRw5G9Uf-aTeKklg~~',
 'ModDate': "D:20220128170914+08'00'",
 'Producer': '; modified using iText® 5.5.13 ©2000-2018 iText Group NV (AGPL-version)',
 'SourceModified': "D:20220117152121+07'21'",
 'Subject': '',
 'Title': '',
 'Trapped': 'False'}

## 文本中的页数

In [13]:
pdf.pages

[<Page:1>, <Page:2>, <Page:3>]

## 页面宽度和高度

In [18]:
pdf.pages[0].width

595.35

In [19]:
pdf.pages[1].height

842

In [1]:
from io import StringIO
from pdfminer.layout import LAParams
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import *
from pdfminer.pdfpage import PDFPage


def convertPDF(fname, pages=None):
    if not pages:
        pagenos = set()
    else:
        pagenos = set(pages)
    caching = True
    outfp = StringIO()
    layoutmode = 'normal'
    laparams = LAParams()
    rotation = 0

    rsrcmgr = PDFResourceManager(caching=caching)
    device = HTMLConverter(rsrcmgr, outfp, scale=1,
                           layoutmode=layoutmode, laparams=laparams,
                           imagewriter=None)
    fp = open(fname, 'rb')
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    for page in PDFPage.get_pages(fp, pagenos,
                                  maxpages=0, password='',
                                  caching=caching, check_extractable=True):
        page.rotate = (page.rotate + rotation) % 360
        interpreter.process_page(page)
    fp.close()
    device.close()

    text = outfp.getvalue()
    outfp.close()
    return text

ModuleNotFoundError: No module named 'pdfminer'

In [2]:
!pip install pdfminer

  Using cached pdfminer-20191125.tar.gz (4.2 MB)
  Using cached pycryptodome-3.14.1-cp35-abi3-win_amd64.whl (1.8 MB)


  Created wheel for pdfminer: filename=pdfminer-20191125-py3-none-any.whl size=6140133 sha256=db12c4c2f10bbfe6da2ed1606826e197e67f03ee31128f9034bae9aea3a3c000
  Stored in directory: c:\users\86183\appdata\local\pip\cache\wheels\1c\28\7d\f390b82bb0307deb63ff27a1474fd308ec68ee028cb9ab6283
Successfully built pdfminer


In [ ]:
def convert_file_to_txt(file):
    """
    文件格式转换及处理
    """

    try:
        # text = parser.from_file(file, service='text')['content']  # 读取各种类型的文件
        if re.search('doc', file) or re.search('docx', file):
            text = parser.from_file(file, service='text')['content']
        elif re.search('jpg', file) or re.search('png', file):
            text = pytesseract.image_to_string(Image.open(file), lang='chi_sim')
        else:
            with pdfplumber.open(file) as pdf:
                text = ''
                for page in pdf.pages:
                    text += page.extract_text() + '\n'

            # with open(file, "rb") as f:
            #     pdf = pdftotext.PDF(f)
            # text = ''
            # for page in pdf:
            #     text += page

                new_text = re.sub(r'\n+', '\n', text)
                if len(new_text) < 100:
                    pdfDoc = fitz.open(file)
                    text = ''
                    for pg in range(pdfDoc.pageCount):
                        page = pdfDoc[pg]
                        rotate = int(0)
                        zoom_x = 2
                        zoom_y = 2
                        mat = fitz.Matrix(zoom_x, zoom_y).prerotate(rotate)
                        pix = page.get_pixmap(matrix=mat, alpha=False)

                        # file_name = re.search('(.*)/(.*).pdf', file).group(2)
                        pix.writePNG('../data/resume_data/resume_pdf_image/' + '%s.png' % pg)  # 将图片写入指定的文件夹内

                        path = "../data/resume_data/resume_pdf_image/%s.png" % pg

                        text += pytesseract.image_to_string(Image.open(path), lang='chi_sim')

                    shutil.rmtree('../data/resume_data/resume_pdf_image')
                    os.mkdir('../data/resume_data/resume_pdf_image')

    except Exception as e:  # Exception常见错误的基类
        logging.error('Error in file:: ' + str(e))
        return [], " "
    try:
        clean_text = re.sub(r'\n+', '\n', text)  # \n+匹配一个或多个换行
        clean_text = clean_text.replace("\r", "\n").replace("\t", " ")  # Normalize text blob，\r回车符，\t制表符
        # splitlines()按照行('\r', '\r\n', \n')分隔，返回一个包含各行作为元素的列表，
        # 如果参数keepends为 False，不包含换行符，如果为True，则保留换行符。
        resume_lines = clean_text.splitlines()  # Split text blob into individual lines
        resume_lines = [re.sub('\s+', ' ', line.strip()) for line in resume_lines if
                        line.strip()]  # Remove empty strings and whitespaces，当不为空时进行处理
        return resume_lines, text  # resume_lines为列表，列表中的元素为简历中的每行内容，text为字符串，为原始简历文本
    except Exception as e:
        logging.error('Error in file:: ' + str(e))
        return [], " "

In [ ]:
html_str_list = convertPDF(resume_name).split('<div')

In [ ]:
layout_vector = []
font_family = {}
font_size = {}
left_loc = {}

resume_lines, full_text = convert_file_to_txt(resume_name)
for txt_line in resume_lines:
    for html_str in html_str_list:
        if txt_line in html_str:
            font_family_tag = re.search('font-family: (.*?);', html_str).group(1)
            if font_family_tag not in font_family:
                font_family[font_family_tag] = 1
            else:
                font_family[font_family_tag] += 1

            font_size_tag = re.search('font-size:(.*?)px', html_str).group(1)
            if font_size_tag not in font_size:
                font_size[font_size_tag] = 1
            else:
                font_size[font_size_tag] += 1

            left_loc_tag = re.search('left:(.*?)px', html_str).group(1)
            if left_loc_tag not in left_loc:
                left_loc[left_loc_tag] = 1
            else:
                left_loc[left_loc_tag] += 1
            break

# 获得字体样式、字体大小和左边位置的最大值
max_font_family = max(font_family, key=font_family.get)
max_font_size = max(font_size, key=font_size.get)
max_left_loc = max(left_loc, key=left_loc.get)

vector_tag = 1

for html_str in html_str_list:
    if line in html_str:
        if re.search('font-family: (.*?);', html_str).group(1) == max_font_family:
            layout_vector.append(1)
        else:
            layout_vector.append(0)

        if re.search('font-size:(.*?)px', html_str).group(1) == max_font_size:
            layout_vector.append(1)
        else:
            layout_vector.append(0)

        if re.search('left:(.*?)px', html_str).group(1) == max_left_loc:
            layout_vector.append(1)
        else:
            layout_vector.append(0)

        vector_tag = 0

        break

if vector_tag:
    layout_vector.append(0)
    layout_vector.append(0)
    layout_vector.append(0)